## Imports & Inputs

In [1]:
from pathlib import Path
from typing import Callable, Iterable, List, Sequence, Tuple

import git
import numpy as np
import numpy.typing as npt


REPO_ROOT = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PUZZLE_INPUT = REPO_ROOT / 'puzzle_inputs' / 'Day 5.txt'

assert PUZZLE_INPUT.exists()

<h2>--- Day 5: Hydrothermal Venture ---</h2><p>You come across a field of <a href="https://en.wikipedia.org/wiki/Hydrothermal_vent" target="_blank">hydrothermal vents</a> on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.</p>
<p>They tend to form in <em>lines</em>; the submarine helpfully produces a list of nearby <span title="Maybe they're Bresenham vents.">lines of vents</span> (your puzzle input) for you to review. For example:</p>
<pre><code>0,9 -&gt; 5,9
8,0 -&gt; 0,8
9,4 -&gt; 3,4
2,2 -&gt; 2,1
7,0 -&gt; 7,4
6,4 -&gt; 2,0
0,9 -&gt; 2,9
3,4 -&gt; 1,4
0,0 -&gt; 8,8
5,5 -&gt; 8,2
</code></pre>
<p>Each line of vents is given as a line segment in the format <code>x1,y1 -&gt; x2,y2</code> where <code>x1</code>,<code>y1</code> are the coordinates of one end the line segment and <code>x2</code>,<code>y2</code> are the coordinates of the other end. These line segments include the points at both ends. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 1,3</code> covers points <code>1,1</code>, <code>1,2</code>, and <code>1,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,7</code> covers points <code>9,7</code>, <code>8,7</code>, and <code>7,7</code>.</li>
</ul>
<p>For now, <em>only consider horizontal and vertical lines</em>: lines where either <code>x1 = x2</code> or <code>y1 = y2</code>.</p>
<p>So, the horizontal and vertical lines from the above list would produce the following diagram:</p>
<pre><code>.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
</code></pre>
<p>In this diagram, the top left corner is <code>0,0</code> and the bottom right corner is <code>9,9</code>. Each position is shown as <em>the number of lines which cover that point</em> or <code>.</code> if no line covers that point. The top-left pair of <code>1</code>s, for example, comes from <code>2,2 -&gt; 2,1</code>; the very bottom row is formed by the overlapping lines <code>0,9 -&gt; 5,9</code> and <code>0,9 -&gt; 2,9</code>.</p>
<p>To avoid the most dangerous areas, you need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is anywhere in the diagram with a <code>2</code> or larger - a total of <code><em>5</em></code> points.</p>
<p>Consider only horizontal and vertical lines. <em>At how many points do at least two lines overlap?</em></p>


### Solution
NumPy makes this dirt simple.

In [2]:
def parse_vent_report(vent_report: str) -> List[List[Tuple[int, int]]]:
    return [
        [
            tuple(map(int, point.split(',')))
            for point in vent.strip().split(' -> ')
        ]
        for vent in vent_report.strip().split('\n')
    ]


def vertical_and_horizontal_vents(vent_list):
    return [
        [(x1, y1), (x2, y2)]
        for [(x1, y1), (x2, y2)] in vent_list
        if x1 == x2 or y1 == y2
    ]


vent_list = parse_vent_report(PUZZLE_INPUT.read_text())
seafloor = np.zeros((1000, 1000), dtype=int)
sign = lambda a, b: 1 if a <= b else -1

for [(x1, y1), (x2, y2)] in vertical_and_horizontal_vents(vent_list):
    seafloor[
        tuple([
            np.arange(x1, x2 + sign(x1, x2), sign(x1, x2)),
            np.arange(y1, y2 + sign(y1, y2), sign(y1, y2))
        ])
    ] += 1

print('Vent Intersections:', (seafloor > 1).sum())

Vent Intersections: 7297


<h2 id="part2">--- Part Two ---</h2><p>Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider <em>diagonal lines</em>.</p>
<p>Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 3,3</code> covers points <code>1,1</code>, <code>2,2</code>, and <code>3,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,9</code> covers points <code>9,7</code>, <code>8,8</code>, and <code>7,9</code>.</li>
</ul>
<p>Considering all lines from the above example would now produce the following diagram:</p>
<pre><code>1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
</code></pre>
<p>You still need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is still anywhere in the diagram with a <code>2</code> or larger - now a total of <code><em>12</em></code> points.</p>
<p>Consider all of the lines. <em>At how many points do at least two lines overlap?</em></p>

### Solution
Part 2 is actually easier than Part 1 with this approach.

In [3]:
seafloor = np.zeros((1000, 1000), dtype=int)
for [(x1, y1), (x2, y2)] in vent_list:
    seafloor[
        tuple([
            np.arange(x1, x2 + sign(x1, x2), sign(x1, x2)),
            np.arange(y1, y2 + sign(y1, y2), sign(y1, y2))
        ])
    ] += 1

print('Vent Intersections:', (seafloor > 1).sum())

Vent Intersections: 21038
